In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True , as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/harshit/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np
train_data , test_data = imdb['train'] , imdb['test']

In [5]:
#training data
training_sentences = []
training_labels = []

#testing data
testing_labels = []
testing_sentences = []

#str(s.tonumpy()) is required and not s.numpy() in python 3

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [6]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [7]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [10]:
tokenizer = Tokenizer(num_words = vocab_size , oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences , maxlen = max_length , truncating = trunc_type)

In [11]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padding = pad_sequences(testing_sequences, maxlen = max_length)

In [12]:
#defining our neural network

model = tf.keras.Sequential(
[
    tf.keras.layers.Embedding(vocab_size, embedding_dim , input_length = max_length),
    tf.keras.layers.Flatten(),         #u can also use GlobalAveragePooling1D()
    tf.keras.layers.Dense(6 , activation = "relu"),
    tf.keras.layers.Dense(1 , activation = "sigmoid")
])

In [13]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 27,533
Trainable params: 27,533
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padding, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 142us/sample - loss: 0.5350 - accuracy: 0.7122 - val_loss: 0.4007 - val_accuracy: 0.8156
Epoch 2/10
25000/25000 [==============================] - 2s 93us/sample - loss: 0.3879 - accuracy: 0.8265 - val_loss: 0.4079 - val_accuracy: 0.8087
Epoch 3/10
25000/25000 [==============================] - 3s 104us/sample - loss: 0.3332 - accuracy: 0.8563 - val_loss: 0.4209 - val_accuracy: 0.8086
Epoch 4/10
25000/25000 [==============================] - 3s 102us/sample - loss: 0.2695 - accuracy: 0.8943 - val_loss: 0.4654 - val_accuracy: 0.7943
Epoch 5/10
25000/25000 [==============================] - 2s 91us/sample - loss: 0.2146 - accuracy: 0.9208 - val_loss: 0.5528 - val_accuracy: 0.7783
Epoch 6/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.1621 - accuracy: 0.9454 - val_loss: 0.6475 - val_accuracy: 0.7721
Epoch 7/10
25000/25000 [=============================

In [16]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [17]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b oh <OOV> <OOV> <OOV> did it again <OOV> baby this movie <OOV> it was one of the <OOV> movies i saw of her and i have to say i feel in love with her she was great in this move br br her performance was <OOV> and what i liked the most was the <OOV> and the <OOV> it was amazing you can tell that they put a lot into the movie the girls <OOV> were amazing br br i hope this comment <OOV> and <OOV> can buy the movie the storyline is <OOV> is very unique and i'm sure <OOV> are going to like it <OOV> <OOV> us once more and no wonder the movie won so many
b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the sto

In [18]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [21]:
sentence = "I really think this is amazing."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1], [968], [4], [1], [1], [1], [], [790], [1], [11], [1], [1], [], [790], [1], [11], [579], [], [11], [579], [], [4], [1], [4], [1], [11], [1], [1], []]


In [23]:
#visualize the data points/vectors at project.tensorflow.org
try:
  from jupyterlab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')